## Model Success Probability Measure

The most intuitive question one can address is on the success rate of the model in generating the correct answer to the prompt. We are able to measure this probability by comparing the output with the truth data. We draw random samples of articles with their truth data from kaggle and run the LLM on each article independently. We then use various distance measures to qunatify the similarity between the output and the truth data. We finally put a threshold level to the results such that if the measurement outcome deviates from this threshold, then we consider the model failed at generating correct outputs. Therefore it is important to know what the threshold should be. The most restrictive threshold will be zero, showing that there is no difference between the outcome and the truth. This is reasonable for research question 1 and 2, however not a good approach for research question 3 because the summary output from the model will most likely have some difference compared to the original abstract of the article. For research question 3, one can choose an reasonable threshold for analysis, but the best approach should be based on the interpretation of a human after having read the summary output from the model and compared it with the abstract.

Once the appropriate measurement strategy is determined, one can conclude whether the model has successfully generated the correct answer to the prompt or not. The result forms a random sample of Bernoulli trials. Suppose $X_1,...,X_n$ are the random sample of the experimental result for a large language model. The Bernoulli random variables have a support on $\{0, 1\}$, with $X_i=1$ if the model generates a correct answer to the prompt with probability $p$, otherwise $X_i=0$ with probability $1-p$. The success rate $p$ is the parameter of the Bernoulli distribution. Our goal is therefore to estimate this success probabilty and compare it between different models.


We can have various hypothesis tests on the success probability $p$. One may consider a right-tailed hypothesis:

$$
H_0: ~ p \leq p_0 \\
H_1: ~ p> p_0
$$

such that we would like to know if the success probability is greater than certain value $p_0$. One may also consider a two-sample hypothesis test:
$$
H_0: ~ p_1 \leq p_2 \\
H_1: ~ p_1 > p_2
$$

such that we would like to know if the success probability of model 1 $p_1$ is greater than the one of model 2 $p_2$. Note that since $p$ is a real number between 0 and 1, it is not informative to havae a two-sided test $H_0: ~ p=0$ VS $H_a: p \neq 0$, so we omit such test.

A general test statistic one can find in most of Statistics textbooks is the normal approximation to the binomial distribution. We can use the likelihood ratio to derive the test statistic for our composite hypothesis:
$$
\Lambda(\vec{x})=\frac{\sup_{p \leq p_0} p_0^{\sum_{i=1}^{n} x_i} (1-p_0)^{n-\sum_{i=1}^{n} x_i} }{\sup_{p \in \Theta}  p^{\sum_{i=1}^{n} x_i} (1-p)^{n-\sum_{i=1}^{n} x_i}}
$$

The maximum likelihood estimator (MLE) of $p$ is $\hat{p}=\frac{1}{n}\sum_{i=1}^{n} x_i$, so $\sum_{i=1}^{n} x_i = n\hat{p}$. The LRT statistic becomes:
$$
\Lambda(\vec{x})=
\begin{cases}
1 & p_0 \geq \frac{y}{n}\\
\frac{(p_0)^{n\hat{p}} \cdot (1-p_0)^{n(1-\hat{p})} }{(\hat{p})^{n\hat{p}} \cdot (1-\hat{p})^{n(1-\hat{p})}} & p_0 < \frac{y}{n}
\end{cases}
$$

Let $g(\vec{x}):=-\ln{\Lambda(\vec{x})} = n\hat{p}(\ln{\hat{p}}-\ln{p_0})+n(1-\hat{p})(\ln{(1-\hat{p})} - \ln{(1-p_0)})$.

By Taylor's expansion, $\ln{(1-\hat{p})} - \ln{(1-p_0)} \approx - \frac{\hat{p}-p_0}{1-p_0}$, $\ln{\hat{p}} - \ln{p_0} \approx \frac{\hat{p}-p_0}{p_0}$. Therefore,
$$
g(\vec{x}) = n(\hat{p}-p_0)\left( \frac{\hat{p}-p_0}{p_0(1-p_0)} \right) = \frac{(\hat{p}-p_0)^2}{p_0(1-p_0)/n}.
$$

By Central Limit Theorem, $T(\vec{x}):=\frac{\hat{p}-p_0}{\sqrt{p_0(1-p_0)/n}}$ is Normal(0,1) distributed. Thus we have that
$$
\Lambda(\vec{x}) < \lambda_0 \equiv T(\vec{x}) > \lambda_0
$$

Our hypothesis test is
$$
\phi(\vec{x}) = 
\begin{cases}
1, \quad T(\vec{x}) > \lambda_0\\
0, \quad T(\vec{x}) \leq \lambda_0
\end{cases}
$$

Using the pivotal quantity $Q(\vec{x}, p):=\frac{\hat{p}-p}{\sqrt{p(1-p)/n}}$ we can get the ($1-\alpha$)-level confidence interval for the success rate as $[L(\vec{X}), R(\vec{X})]$ such that:
$$
\mathbb{P}(p \in [L(\vec{X}), R(\vec{X})])\geq 1-\alpha
$$

The interpretation of the confidence interval is important. In a frequentist point of view, our unknown parameter $p$ is a true but fixed value. The confidence interval on the other hand is a random set, thus it is incorrect to state that we are 99% sure that the confidence interval covers the unknown true paramter. It is also incorrect to say that the unknown parameter falls within the confidence interval 99% of the time, because in this statement the unknown parameter is falsely treated as a random variable. Therefore the appropriate interpretation of the confidence interval is: when the experiment is repeated many times to generate random samples and their perspective confidence intervals, 99% of these confidence intervals will cover the unknown fixed parameter.

Alternatively, we can use the Bayesian approach to gain a different understanding of our unknown parameter and to draw inference on it. Assuming we have a prior knowledge of this success rate, say 80%, before observing the data. The Baysian approach allows us to "correct" our degree of beliefs based on the actual observation. In particular, the Bayes' Theorem says that 
$$
P(A|B) = \frac{P(A) \cdot P(B|A)}{P(B)}
$$
where $P(A)$ is the prior probability reflecting our prior belief and $P(A|B)$ is the posterior probability after having observed the event B. Such analysis allows us to make an inference on the success rate in a very different point of view, which we will explain later.

Suppose we assume $Beta(\alpha, \beta)$ is the prior distribution of the success rate p. The prior density function is:
$$
f(p; \alpha,\beta) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} p^{\alpha-1} (1-p)^{\beta-1}, ~ p\in (0,1)
$$
We know the expectation and the variance of Beta distribution as:
$$
\begin{gather*}
E(p)= \frac{\alpha}{\alpha+\beta}\\
Var(p)= \frac{\alpha \beta}{(\alpha+\beta)^2(\alpha+\beta+1)}
\end{gather*}
$$
Thus if we assume that the average success rate is 0.6 with a standard deviation of 0.2, then we can obtain the parameters of the prior distribution as $\alpha = 3, \beta=2$. 

Since the random sample $X_1,...,X_n$ follows Bernoulli(p) distribution, we have a conjugate pair of the prior and posterior distributions:
$$
\begin{align*}
f(p|\vec{x})&=\frac{f(p)f(\vec{x}|p)}{\int_0^1 f(p)f(\vec{x}|p) dp}=\frac{\frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} p^{\alpha-1+\sum_{i=1}^n x_i} (1-p)^{\beta-1+n-\sum_{i=1}^n x_i}}{\int_0^1 \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} p^{\alpha-1+\sum_{i=1}^n x_i} (1-p)^{\beta-1+n-\sum_{i=1}^n x_i} dp}=\frac{p^{\alpha-1+\sum_{i=1}^n x_i} (1-p)^{\beta-1+n-\sum_{i=1}^n x_i}}{\frac{\Gamma(\alpha+\sum_{i=1}^n x_i)\Gamma(\beta+n-\sum_{i=1}^n x_i)}{\Gamma(\alpha+\beta+n)}}\\
&=\frac{\Gamma(\alpha+\beta+n)}{\Gamma(\alpha+\sum_{i=1}^n x_i)\Gamma(\beta+n-\sum_{i=1}^n x_i)} p^{\alpha-1+\sum_{i=1}^n x_i} (1-p)^{\beta-1+n-\sum_{i=1}^n x_i}
\end{align*}
$$
i.e. the posterior distribution is $Beta(\alpha+\sum_{i=1}^n x_i, \beta+n-\sum_{i=1}^n x_i)$ with the mean and variance:
$$
\begin{gather*}
E(p|\vec{x})=\frac{\alpha+\sum_{i=1}^n x_i}{\alpha+\beta+n}\\
Var(p|\vec{x})=\frac{(\alpha+\sum_{i=1}^n x_i)(\beta+n-\sum_{i=1}^n x_i)}{(\alpha+\beta+n)^2(\alpha+\beta+n+1)}
\end{gather*}
$$
One can see that asymptotically when we have a large sample size n, the mean of the posterior distribution approaches to the maximum likelihood estimator $\hat{p} =\frac{1}{n}\sum_{i=1}^n x_i$. Thus the specification of a prior is less important if the sample size is large.  This allows us to make a subjective assumption of the success rate based on a probability distribution instead of viewing it as a fixed but unknown number in the frequentist hypothesis test. In the case where $\alpha=\beta=1$, we obtain the non-informative prior $Beta(1,1)$, which is $Uniform([0,1])$ distribution. It's called non-informative because we assume that all p are equally probable. In other words, we no longer put a subjective assumption on $p$. 

After obtaining the posterior distribution, we can get the ($1-\alpha$)-level credible interval for the success rate based on the quantile of the posterior as $[L(\vec{x}), R(\vec{x})]$ such that:
$$
\mathbb{P}(\bold{p} \in [L(\vec{x}), R(\vec{x})] |\vec{x}) = \int_{L(\vec{x})}^{R(\vec{x})} f(p|\vec{x})dp = 1-\alpha
$$
With such definition, it is perfectly fine to interpret the credible interval as: the probability that our success rate falls within the given credible interval is (1-$\alpha$)%. 

Various research papers have shown that the Bayesian Uniform prior yields better estimation of the confidence interval than the one obtained from the normal approximation of the binomial distribution. We can make a simulation to justify this.


In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import plotly.express as px
from ModelEvaluation import Bayes


In [2]:
def effCP(n, p, alpha, beta, alphalevel=0.05):
    k=np.linspace(0,n,n+1)
    param_1 = (alpha+np.array(k)).tolist()
    param_2 = (beta+n-np.array(k)).tolist()
    #DataF=pd.DataFrame({'k':k, 'param_1':param_1, 'param_2': param_2})
    
    CI_Wald=[[x/n + stats.norm.ppf(alphalevel/2)*math.sqrt((x/n)*(1-x/n)/n), x/n - stats.norm.ppf(alphalevel/2)*math.sqrt((x/n)*(1-x/n)/n)] for x in k]
    CI_Bayes=[stats.beta.ppf([alphalevel/2, 1-alphalevel/2], x,y) for x, y in zip(param_1, param_2)]
    
    Covered_Wald=[1 if p >= x[0] and p <= x[1] else 0 for x in CI_Wald]
    Covered_Bayes=[1 if p >= x[0] and p <= x[1] else 0 for x in CI_Bayes]

    CP_Wald=Covered_Wald*stats.binom.pmf(k, n, p)
    CP_Bayes=Covered_Bayes*stats.binom.pmf(k, n, p)
    return([sum(CP_Wald), sum(CP_Bayes)])


def MeanEffCov(n, p, alpha, beta, alphalevel=0.05):
    effData=[]
    for i in p:
        effData.append(effCP(n, i, alpha, beta, alphalevel))
    effData=pd.DataFrame(effData,columns=['CP_Wald','CP_Bayes'])
    return([effData['CP_Wald'].mean(), effData['CP_Bayes'].mean()])


def CI_z(data, threshold, alphalevel):
    n=len(data)
    p_hat=len(data[data<=threshold])/n
    CI=[p_hat + stats.norm.ppf(alphalevel/2)*math.sqrt(p_hat*(1-p_hat)/n), p_hat - stats.norm.ppf(alphalevel/2)*math.sqrt(p_hat*(1-p_hat)/n)]
    CI=["{:.2%}".format(i) for i in CI]
    print("The maximum likelihood estimate is: " + str("{:.2%}".format(p_hat)))
    print(f"The 99%-level confidence interval for {data.name} distance measure is: " + str(CI))

In [3]:
n=15
p=np.linspace(0.025,0.975,1000)
effData=[]
for proportion in p:
    effData.append(effCP(n, proportion, 1, 1, 0.05))
effData=pd.DataFrame(effData,columns=['CP_Wald','CP_Bayes'])
effData.insert(0,'p', p)
effData

,p,CP_Wald,CP_Bayes
0,0.025000,0.315552,0.947106
1,0.025951,0.325426,0.943468
2,0.026902,0.335159,0.939743
3,0.027853,0.344752,0.935932
4,0.028804,0.354205,0.932038
...,...,...,...
995,0.971196,0.354205,0.932038
996,0.972147,0.344752,0.935932
997,0.973098,0.335159,0.939743
998,0.974049,0.325426,0.943468


In [4]:
n=list(range(10,350,10))
p=np.linspace(0.01,0.99,200)
alpha=1
beta=1
alphalevel=0.05

MeanEffData=[]
for size in n:
    MeanEffData.append(MeanEffCov(size, p, alpha, beta, alphalevel))

MeanEffData=pd.DataFrame(MeanEffData,columns=['CP_Wald','CP_Bayes'])
MeanEffData.insert(0,'n', n)
MeanEffData=MeanEffData.reset_index()
MeanEffData=pd.melt(MeanEffData, id_vars='n', value_vars=['CP_Wald', 'CP_Bayes'])

/Users/tengli/Python/Bayesian_Binomial/BayesEnv/lib/python3.11/site-packages/scipy/stats/_discrete_distns.py:77: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)
/Users/tengli/Python/Bayesian_Binomial/BayesEnv/lib/python3.11/site-packages/scipy/stats/_discrete_distns.py:77: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)


In [5]:
fig=px.line(MeanEffData, x='n', y='value', color='variable',
            labels=dict(n="Sample Size", value="Coverage Probability", variable="Method"))
arr=np.arange(0, 1, 0.01)
fig.update_layout(yaxis=dict(tickmode='array', tickvals=arr, ticktext=[str(round(x,4)) for x in arr]),
                  xaxis=dict(tickmode='linear', tick0=0, dtick=20),
                  width=1200, height=700,
                  title=dict(text=f"Mean Coverage Probability for 95% Confidence Level", x=0.5)
                  )
fig.add_hline(y=0.95,line_dash="dash", line_color="black", line_width=0.5)
fig.show()

In [6]:
x=[(f"sim_p:{p}", stats.bernoulli.rvs(p, size=60)) for p in [0.8342,0.21,0.72,0.97,0.05,0.4]]
x=dict(x)

threshold=[1]*6
alphalevel=0.05
alpha=[1]*6
beta=[1]*6

result=Bayes(x, threshold, alpha, beta, alphalevel, direction='greater')
display(result.res)
result.Plot_Bayes()

,Component,Expected Success Probability,Variance of Success Probability,95.0%-level Credible Interval
0,sim_p:0.8342,0.7903,0.002630,"[0.6816, 0.8814]"
1,sim_p:0.21,0.1613,0.002147,"[0.0815, 0.2617]"
2,sim_p:0.72,0.7097,0.003270,"[0.5917, 0.8148]"
3,sim_p:0.97,0.9355,0.000958,"[0.8629, 0.9818]"
4,sim_p:0.05,0.0806,0.001177,"[0.0272, 0.1595]"
5,sim_p:0.4,0.4677,0.003952,"[0.346, 0.5915]"


In [7]:
# Use weakly informative prior
threshold=[1]*6
alphalevel=0.05
alpha=[1,1,1,1,   9429/800,  12] #the customized prior has mean 0.035 and std 0.01 for the 5th p, and 0.5, 0.2 for the 6th p
beta=[1,1,1,1,   259971/800,  12]

result=Bayes(x, threshold, alpha, beta, alphalevel, direction='greater')
display(result.res)
result.Plot_Bayes()

,Component,Expected Success Probability,Variance of Success Probability,95.0%-level Credible Interval
0,sim_p:0.8342,0.7903,0.002630,"[0.6816, 0.8814]"
1,sim_p:0.21,0.1613,0.002147,"[0.0815, 0.2617]"
2,sim_p:0.72,0.7097,0.003270,"[0.5917, 0.8148]"
3,sim_p:0.97,0.9355,0.000958,"[0.8629, 0.9818]"
4,sim_p:0.05,0.0398,0.000096,"[0.0229, 0.0611]"
5,sim_p:0.4,0.4762,0.002935,"[0.3708, 0.5826]"
